alpha-mind的portfolio文件夹提供了构建组合的工具函数。
- 不同的构建方式实质上对应了不同的优化问题。
- 该文件夹报告的构建方式包括线性构建、多空构建

### 线性构建： *linear_builder*
- 目标: 在线性约束下最大化组合的预期收益率。
- 线性约束包括
  - 标的权重的上下限 *lbound, ubound*
  - 组合风险暴露的上下限 *risk_target* : 函数内部会以因子矩阵乘以权重得到组合的风险暴露。 
  - 仓位调整的换手率上限(换手率以目标权重和当前权重向量的一范数表示，即差的绝对值之和) *turn_over_target*

In [16]:
import numpy as np
from alphamind.portfolio.linearbuilder import linear_builder

"""
问题初始化
"""

# 假设有5个标的
n = 5

# 假设当前标的的权重
current_pos = np.random.randint(0, n, size=n)
current_pos = current_pos / current_pos.sum() 

# 假设标的预期收益向量
expect_return = np.random.randn(n)

# 假设标的风险因子矩阵, 仅有一个风险因子
risk_factors = np.ones((n, 1))

# 约束条件
# 约束条件1 - 标的权重的上下限
weight_lb = np.zeros(n)
weight_ub = 0.5 * np.ones(n)

# 限制条件2 - 组合风险暴露的上下限
risk_lbound = np.ones(1)
risk_ubound = np.ones(1)

# 限制条件3 - 仓位调整的换手率上限（下限为0，故无需设置）
turn_over_target = 0.1

"""
问题求解
"""
status, fvalue, x_values = linear_builder(expect_return,
                                          weight_lb,
                                          weight_ub,
                                          risk_factors,
                                          (risk_lbound, risk_ubound),
                                          turn_over_target,
                                          current_pos,
                                          method='ecos')


print('Optimization status - {}'.format(status))
print('Optimal expect return - {}'.format(fvalue))
print('Optimial portfolio weights - {}'.format(x_values))
print('Initial portfolio weights - {}'.format(current_pos))
print('Turn over amount - {}'.format(np.abs(x_values - current_pos).sum()))

Optimization status - optimal
Optimal expect return - 0.4800809979552383
Optimial portfolio weights - [-5.03898456e-14  7.50000000e-02  1.75000000e-01  2.50000000e-01
  5.00000000e-01]
Initial portfolio weights - [0.    0.125 0.125 0.25  0.5  ]
Turn over amount - 0.10000000000010972


### 多空构建: *long_short_builder*
- 根据给定组数*(n_bins)*，按从小达到的顺序进行分组，返回每个因子属于的组别。